# Information

**Author:**<br>Pascal Munaretto (<a href="mailto:pascal.munaretto@outlook.com">Mail</a>)

**Date:**<br>30.09.2022

**Type:**<br>Master's Thesis

**Topic:**<br>Design, Implementation and Performance Analysis of an AI-Based Insider Threat Detection Platform	in Splunk To Counteract Data Exfiltration

**Study Program:**<br>Enterprise and IT Security

**Institution:**<br><a href="https://www.hs-offenburg.de">Offenburg University of Applied Sciences</a>

**Github:**<br>https://github.com/pmunaretto/Master-Thesis

# Setup

## Requirements

In [ ]:
!pip install pyod suod

## Patches

In [ ]:
# Add callbacks to Auto Encoder, VAE and Deep SVDD
!cp /content/drive/MyDrive/CERT/patches/patched_auto_encoder.py /usr/local/lib/python3.7/dist-packages/pyod/models/auto_encoder.py
!cp /content/drive/MyDrive/CERT/patches/patched_vae.py /usr/local/lib/python3.7/dist-packages/pyod/models/vae.py
!cp /content/drive/MyDrive/CERT/patches/patched_deep_svdd.py /usr/local/lib/python3.7/dist-packages/pyod/models/deep_svdd.py

## Imports

In [ ]:
import os
import math
import sys
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from timeit import default_timer as timer
from random import seed, randint
from sklearn.base import TransformerMixin, BaseEstimator, clone
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import ParameterGrid
from sklearn.preprocessing import RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import roc_auc_score, recall_score, f1_score, precision_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.compose import make_column_selector as selector
from matplotlib.backends.backend_pgf import FigureCanvasPgf
from matplotlib.ticker import PercentFormatter
from pyod.models.iforest import IForest
from pyod.models.ecod import ECOD
from pyod.models.copod import COPOD
from pyod.models.loda import LODA
from pyod.models.cblof import CBLOF
from pyod.models.pca import PCA
from pyod.models.auto_encoder import AutoEncoder
from pyod.models.vae import VAE
from pyod.models.deep_svdd import DeepSVDD
from IPython.display import display, Markdown

## Configuration

In [ ]:
matplotlib.backend_bases.register_backend("pgf", FigureCanvasPgf)

plt.rcParams.update({
    "figure.dpi": 100,
    "savefig.dpi": 300,
    "font.size": 12,
    "image.cmap": "plasma",
    "axes.prop_cycle": plt.cycler("color", "bgrcmyk"), 
    "pgf.texsystem": "pdflatex",
    "font.family": "serif",
    "text.usetex": True,
    "pgf.rcfonts": False
})

tf.get_logger().setLevel("WARN")

# Global Configuration
BASE_PATH     = "/content/drive/MyDrive/CERT/r4.2"
N_JOBS        = -1
N_ITER        = 5
CONTAMINATION = 0.01
RETRAIN       = False
DATASET_NAME  = "logon_single_user"
FEATURE_SETS  = [
    ["activity", "hour"],
    ["activity", "weekday"],
    ["activity", "hour", "weekday"],
    ["activity", "hour_sin", "hour_cos"],
    ["activity", "weekday_sin", "weekday_cos"],
    ["activity", "hour_sin", "hour_cos", "weekday_sin", "weekday_cos"]
]

## Helper Functions

In [ ]:
class Debugger(BaseEstimator, TransformerMixin):

    def transform(self, data):
        print("Shape of Preprocessed data:", data.shape)
        print(pd.DataFrame(data).head())
        return data

    def fit(self, data, y=None, **fit_params):
        return self


def plot_anomaly_scores(series, identifier, min, max, save=True):
    plt.figure(figsize=(10,3))
    plt.hist(
        series,
        weights=np.ones(len(series)) / len(series),
        bins=np.arange(min, max, 0.02),
        rwidth=0.8
    )
    plt.xlim(xmin=min, xmax=max)
    plt.xticks(np.arange(min, max+0.1, 0.1))
    plt.xlabel("Anomaly Score")
    plt.ylabel("Percentage")
    plt.gca().yaxis.set_major_formatter(PercentFormatter(1))
    if save:
        plt.savefig(os.path.join(BASE_PATH, "figures", f"{identifier}.pgf"), format="pgf")
    plt.show()


def plot_confusion_matrix(y_true, y_pred, identifier, save=True):
    ConfusionMatrixDisplay.from_predictions(
        y_true,
        y_pred,
        labels=[0, 1],
        display_labels=["Benign", "Malicious"],
        values_format="d",
        colorbar=True,
        cmap="plasma_r"
    )
    plt.grid(False)
    if save:
        plt.savefig(os.path.join(BASE_PATH, "figures", f"{identifier}.pgf"), format="pgf")
    plt.show()


def print_training_result(metrics):
    print(
        "  ".join(
            [
                f"\033[1;33m Training Time: {metrics.training_time_avg:<7.4f}\033[0m",
                f"\033[1;33m Inference Time: {metrics.inference_time_avg:<7.4f}\033[0m",
                f"\033[1;35m pAUC: {metrics.p_auc_10_avg:02.4f} \u00B1 {metrics.p_auc_10_std:02.4f}\033[0m",
                f"\033[1;35m Recall: {metrics.recall_avg:02.4f} \u00B1 {metrics.recall_std:02.4f}\033[0m",
                f"\033[1;32m TN: {metrics.best_classifier_TN:<6}\033[0m",
                f"\033[1;31m FP: {metrics.best_classifier_FP:<5}\033[0m",
                f"\033[1;31m FN: {metrics.best_classifier_FN:<3}\033[0m",
                f"\033[1;32m TP: {metrics.best_classifier_TP:<3}\033[0m",
                f"\033[1;37m Params: {metrics.name}\033[0m"
            ]
        )
    )


def print_gridsearch_result(metrics):
    print(
        "\n".join(
            [
                "\n\033[4mBest hyperparameters:\033[0m",
                f"Params: {metrics.name}",
                f"pAUC:   {metrics.p_auc_10_avg:02.4f} \u00B1 {metrics.p_auc_10_std:02.4f}",
                f"Recall: {metrics.recall_avg:02.4f} \u00B1 {metrics.recall_std:02.4f}"
            ]
        )
    )


def calculate_dispersion_metrics_for_columns(source_df, destination_df, columns):
    for column in columns:
        avg = np.average(source_df[column])
        std = np.std(source_df[column])
        destination_df[f"{column}_avg"] = avg
        destination_df[f"{column}_std"] = std if not math.isnan(std) else 0

    return destination_df


def train_classifier_on_single_users(df, classifier, features, params, n_iter=10):
    # Create a dataframe where the results of the different seeds will be stored
    random_state_summary = pd.DataFrame()

    # Reset the PRNG seed
    seed(1)

    # Group the dataframe by users
    grouped = df.groupby("user", as_index=False)

    # Perform the training process multiple times with random seeds
    for _ in range(n_iter):

        # Create a dataframe where the results of the classifiers will be stored
        user_timings = pd.DataFrame()
        user_predictions = pd.DataFrame()

        for name, group in grouped:

            print(name)

            # Create a clone of the classifier
            try:
                classifier = clone(classifier)
            except:
                pass
            
            # Update the parameters of the classifier according to the grid search
            classifier.set_params(**params)
            
            # Set the random state attribute of the classifier (if it has one)
            try:
                classifier.set_params(**{"random_state": randint(0, 2**32)})
            except Exception:
                pass

            # Define the transformers that do the rest of the preprocessing (scaling, encoding)
            numeric_transformer = Pipeline(steps=[
                ("scaler", RobustScaler())
            ])
            categorical_transformer = Pipeline(steps=[
                ("ohe", OneHotEncoder())
            ])

            # Filter the dataframe according to the features 
            group_filtered = group[features].copy()

            # Create a pipeline that performs the feature selection and scaling
            pipe = Pipeline([
                ("column_transformer", ColumnTransformer(
                    transformers=[
                        ("num", numeric_transformer, selector(dtype_exclude=["category", "object"])),
                        ("cat", categorical_transformer, selector(dtype_include=["category", "object"]))
                    ]
                )),
                ("classifier", classifier)
            ])
        
            # Benchmark the training
            start_training = timer()
            pipe.fit(group_filtered)
            end_training = timer()

            # Benchmark the inference
            start_inference = timer()
            pipe.predict(group_filtered)
            end_inference = timer()

            # Add the predictions and anomaly scores to the group dataframe
            group["y_true"] = group["threat"]
            group["y_pred"] = pipe.named_steps["classifier"].labels_
            group["scores"] = pipe.named_steps["classifier"].decision_scores_

            # Create a new series with the training metrics for the user iteration
            timings = pd.Series(
                {
                    "training_time": end_training - start_training,
                    "inference_time": end_inference - start_inference
                }
            )

            # Append the series to our user summary dataframes
            user_timings = user_timings.append(timings, ignore_index=True)
            user_predictions = pd.concat([user_predictions, group])

        # If there are inf values in the anomaly scores, replace it with the elsewise highest value
        max_without_infs = user_predictions["scores"].replace([np.inf, -np.inf], np.nan).max()
        user_predictions["scores"].replace(np.inf, max_without_infs, inplace=True)
        user_predictions["scores"].replace(np.nan, 0, inplace=True)

        # Find the right threshold to satisfy the contamination and add the final predictions
        upper = max_without_infs
        lower = 0
        for i in range(0,500):
            threshold = (upper + lower) / 2
            test = (user_predictions.scores > threshold).astype(bool)
            if test.sum() > CONTAMINATION * len(df):
                lower = threshold
            else:
                upper = threshold
        user_predictions["y_pred"] = test

        # After the predictions were added, calculate the evaluation metrics
        recall = recall_score(user_predictions.threat, user_predictions.y_pred)
        precision = precision_score(user_predictions.threat, user_predictions.y_pred)
        f1 = f1_score(user_predictions.threat, user_predictions.y_pred)
        cm = confusion_matrix(user_predictions.threat, user_predictions.y_pred, labels=[0, 1])

        # Try to calculate the AUC, this could fail if one of the anomaly scores is infinite
        try:
            auc = roc_auc_score(user_predictions.threat, user_predictions.scores)
            p_auc_10 = roc_auc_score(user_predictions.threat, user_predictions.scores, max_fpr=0.1)
            p_auc_20 = roc_auc_score(user_predictions.threat, user_predictions.scores, max_fpr=0.2)
            p_auc_30 = roc_auc_score(user_predictions.threat, user_predictions.scores, max_fpr=0.3)
        except ValueError as e:
            print(e)

        # Create a new series with all the information about the iteration
        metrics = pd.Series(
            {
                "training_time": user_timings.training_time.sum(),
                "inference_time": user_timings.inference_time.sum(),
                "recall": recall,
                "precision": precision,
                "f1": f1,
                "TN": cm[0][0],
                "FP": cm[0][1],
                "FN": cm[1][0],
                "TP": cm[1][1],
                "auc": auc,
                "p_auc_10": p_auc_10,
                "p_auc_20": p_auc_20,
                "p_auc_30": p_auc_30,
                "y_true": user_predictions.y_true,
                "y_pred": user_predictions.y_pred,
                "scores": user_predictions.scores,
            }
        )

        # Append the series to our summary dataframe
        random_state_summary = random_state_summary.append(metrics, ignore_index=True)

    # Convert the confusion matrix to integers
    random_state_summary = random_state_summary.astype({"TN": "int32", "FP": "int32", "FN": "int32", "TP": "int32"})

    # Locate the best classifier and separate the predictions from it
    results = random_state_summary.iloc[random_state_summary.p_auc_10.argmax()].rename(str(params))
    predictions = results.loc[["y_true", "y_pred", "scores"]]

    # Remove the columns that should not be part of the results dataframe
    results.drop(["y_true", "y_pred", "scores"], inplace=True)

    # Add the prefix
    results = results.add_prefix("best_classifier_")
    
    # Add the average training and inference time to the dataframe
    results["training_time_avg"]  = np.average(random_state_summary["training_time"])
    results["inference_time_avg"] = np.average(random_state_summary["inference_time"])

    # Calculate averages and different dispersion metrics for the best classifier series
    results = calculate_dispersion_metrics_for_columns(
        source_df=random_state_summary,
        destination_df=results,
        columns=["auc", "p_auc_10", "p_auc_20", "p_auc_30", "recall"]
    )

    return results, predictions


class GridSearch:
    def __init__(self, df, classifier, features, parameters, gridsearch_path):

        # Instance variables
        self.df = df
        self.classifier = classifier
        self.features = features
        self.parameters = parameters
        self.gridsearch_path = gridsearch_path

        # Main paths
        self.summary_path = os.path.join(self.gridsearch_path, "gridsearch_summary.csv")
        self.best_results_path = os.path.join(self.gridsearch_path, "best_results.csv")
        self.best_preds_path = os.path.join(self.gridsearch_path, "best_preds.csv")

        # Create the output directory for the gridsearch
        os.makedirs(gridsearch_path, exist_ok=True)

        # Read existing files
        if os.path.exists(self.summary_path) and not RETRAIN:
            self.gridsearch_summary = pd.read_csv(self.summary_path, index_col=0)
        else:
            self.gridsearch_summary = pd.DataFrame()
        if os.path.exists(self.best_results_path) and not RETRAIN:
            self.best_results = pd.read_csv(self.best_results_path, squeeze=True, index_col=0)
        else:
            self.best_results = None


    def start_training(self):

        # Create an iterable parameter grid from the parameters dictionary
        grid = ParameterGrid(self.parameters)

        # Debug output
        print(f"\033[4mTesting {len(list(grid))} different hyperparameter combinations\033[0m")

        # Iterate over all possible parameter combinations
        for params in grid:

            # Skip the parameters if they are already part of the gridsearch summary
            if not RETRAIN and not self.gridsearch_summary.empty and str(params) in self.gridsearch_summary.index:
                print_training_result(self.gridsearch_summary.loc[str(params)])
                continue

            # Start the training process
            try: 
                if hasattr(classifier, "random_state"):
                    results, predictions = train_classifier_on_single_users(
                        df=self.df,
                        classifier=self.classifier,
                        features=self.features,
                        params=params,
                        n_iter=N_ITER
                    )
                else:
                    results, predictions = train_classifier_on_single_users(
                        df=self.df,
                        classifier=self.classifier,
                        features=self.features,
                        params=params,
                        n_iter=1
                    )
            except ValueError as e:
                print(f"Skipping {params}: {e}")
                continue

            # Print the metrics of the best classifier
            print_training_result(results)

            # Add the results to the gridsearch summary
            self.gridsearch_summary = self.gridsearch_summary.append(results)

            # Update the best classifier if the iterations performs better than the current best 
            if self.best_results is None or results.p_auc_10_avg > self.best_results.p_auc_10_avg:
                self.best_results = results
                self.best_preds = predictions
                self.save_best_results()

            # Save the progress
            self.save_gridsearch_summary()

        # Print the results of the gridsearch (parameters with the best average)
        print_gridsearch_result(self.best_results)

        return self.best_results


    def save_gridsearch_summary(self):
        self.gridsearch_summary.to_csv(self.summary_path)


    def save_best_results(self):
        self.best_results.to_csv(self.best_results_path)
        self.best_preds.to_frame()\
            .transpose()\
            .apply(pd.Series.explode)\
            .reset_index(drop=True)\
            .to_csv(self.best_preds_path, index=False)


    def get_summary(self):
        return self.gridsearch_summary


def initiate_training_run(classifier_name, classifier, parameters):
    # Define output paths
    summary_path = os.path.join(BASE_PATH, "results_summary", DATASET_NAME)
    summary_file = os.path.join(summary_path, "summary.csv")

    # Create the output directory for the classifier
    os.makedirs(summary_path, exist_ok=True)

    # Iterate through the feature sets
    for i, features in enumerate(FEATURE_SETS, start=1):

        display(Markdown(f"# {i}/{len(FEATURE_SETS)} - Features: {', '.join(features)}"))

        # Perform a grid search to find the best parameters for the classifier
        gridsearch = GridSearch(
            df=df,
            classifier=classifier,
            features=features,
            parameters=parameters,
            gridsearch_path=os.path.join(BASE_PATH, "results_summary", DATASET_NAME, classifier_name, f"gridsearch{i}")
        )
    
        best_parameter_series = gridsearch.start_training()

        # Read the summary file if it already exists, otherwise create a new one
        if os.path.exists(summary_file):
            summary = pd.read_csv(summary_file, index_col=0)
        else:
            summary = pd.DataFrame()

        # Set the index of the pandas series and update / append it to the summary
        index_name = f"{classifier_name}_dataset{i}"
        best_parameter_series.rename(index_name, inplace=True)
        if index_name in summary.index:
            summary.loc[index_name] = best_parameter_series
        else:
            summary = summary.append(best_parameter_series)

        # Save the summaries and predictions to a file
        summary.sort_index(inplace=True)
        summary.to_csv(summary_file)

## Loading the Data

In [ ]:
# Read the dataset
df = pd.read_parquet(os.path.join(BASE_PATH, "preprocessed", "logon"))

# Training - Isolation Forest

In [ ]:
# Local configuration
classifier_name = "isolation_forest"

# Define the classifier that will be used for training
classifier = IForest(
    behaviour="new",
    max_features=1.0,
    contamination=CONTAMINATION,
    n_jobs=N_JOBS
)

# Define the hyperparameters grid that will be tested for best results
parameters = {
    "n_estimators": [1, 10, 50, 100],
    "max_samples": [128, 256, 512, 1024, 2048, 4096],
}

# Start the training
initiate_training_run(classifier_name, classifier, parameters)

# 1/6 - Features: activity, hour

Testing 24 different hyperparameter combinations
 Training Time: 12.7451   Inference Time: 4.2983    pAUC: 0.6198 ± 0.0121   Recall: 0.1990 ± 0.0213   TN: 853574.0   FP: 1087.0   FN: 154.0   TP: 44.0   Params: {'max_samples': 128, 'n_estimators': 1}
 Training Time: 119.9089   Inference Time: 8.2144    pAUC: 0.6709 ± 0.0079   Recall: 0.3657 ± 0.0182   TN: 851983.0   FP: 2678.0   FN: 119.0   TP: 79.0   Params: {'max_samples': 128, 'n_estimators': 10}
 Training Time: 153.4035   Inference Time: 24.9018   pAUC: 0.6787 ± 0.0066   Recall: 0.3889 ± 0.0132   TN: 851908.0   FP: 2753.0   FN: 117.0   TP: 81.0   Params: {'max_samples': 128, 'n_estimators': 50}
 Training Time: 295.0262   Inference Time: 45.4202   pAUC: 0.6826 ± 0.0073   Recall: 0.3970 ± 0.0145   TN: 851946.0   FP: 2715.0   FN: 115.0   TP: 83.0   Params: {'max_samples': 128, 'n_estimators': 100}
 Training Time: 11.9028   Inference Time: 3.9291    pAUC: 0.6579 ± 0.0193   Recall: 0.2737 ± 0.0450   TN: 852824.0   FP: 1837.0   FN: 130.0 

# 2/6 - Features: activity, weekday

Testing 24 different hyperparameter combinations
 Training Time: 14.9347   Inference Time: 5.1148    pAUC: 0.5122 ± 0.0077   Recall: 0.0192 ± 0.0049   TN: 854438.0   FP: 223.0   FN: 194.0   TP: 4.0   Params: {'max_samples': 128, 'n_estimators': 1}
 Training Time: 126.6483   Inference Time: 11.0947   pAUC: 0.5303 ± 0.0085   Recall: 0.0606 ± 0.0085   TN: 854227.0   FP: 434.0   FN: 184.0   TP: 14.0   Params: {'max_samples': 128, 'n_estimators': 10}
 Training Time: 250.6031   Inference Time: 36.9798   pAUC: 0.5252 ± 0.0036   Recall: 0.0687 ± 0.0025   TN: 854177.0   FP: 484.0   FN: 184.0   TP: 14.0   Params: {'max_samples': 128, 'n_estimators': 50}
 Training Time: 336.6046   Inference Time: 61.2960   pAUC: 0.5272 ± 0.0030   Recall: 0.0707 ± 0.0000   TN: 854169.0   FP: 492.0   FN: 184.0   TP: 14.0   Params: {'max_samples': 128, 'n_estimators': 100}
 Training Time: 14.3311   Inference Time: 4.6850    pAUC: 0.5204 ± 0.0070   Recall: 0.0384 ± 0.0069   TN: 854334.0   FP: 327.0   FN: 189.0   TP: 

# 3/6 - Features: activity, hour, weekday

...

Testing 24 different hyperparameter combinations
 Training Time: 16.4153   Inference Time: 5.9013    pAUC: 0.6134 ± 0.0064   Recall: 0.1859 ± 0.0190   TN: 853543.0   FP: 1118.0   FN: 157.0   TP: 41.0   Params: {'max_samples': 128, 'n_estimators': 1}
 Training Time: 127.4473   Inference Time: 11.4512   pAUC: 0.6696 ± 0.0101   Recall: 0.3636 ± 0.0209   TN: 851462.0   FP: 3199.0   FN: 119.0   TP: 79.0   Params: {'max_samples': 128, 'n_estimators': 10}
 Training Time: 185.6521   Inference Time: 34.2472   pAUC: 0.6693 ± 0.0054   Recall: 0.3667 ± 0.0155   TN: 851327.0   FP: 3334.0   FN: 122.0   TP: 76.0   Params: {'max_samples': 128, 'n_estimators': 50}
 Training Time: 334.3841   Inference Time: 62.7243   pAUC: 0.6680 ± 0.0031   Recall: 0.3626 ± 0.0074   TN: 851265.0   FP: 3396.0   FN: 124.0   TP: 74.0   Params: {'max_samples': 128, 'n_estimators': 100}
 Training Time: 14.6963   Inference Time: 4.7795    pAUC: 0.6379 ± 0.0118   Recall: 0.2323 ± 0.0300   TN: 852852.0   FP: 1809.0   FN: 143.0 

# 5/6 - Features: activity, weekday_sin, weekday_cos

Testing 24 different hyperparameter combinations
 Training Time: 15.9035   Inference Time: 5.4624    pAUC: 0.5136 ± 0.0100   Recall: 0.0242 ± 0.0038   TN: 854412.0   FP: 249.0   FN: 193.0   TP: 5.0   Params: {'max_samples': 128, 'n_estimators': 1}
 Training Time: 125.9430   Inference Time: 10.6391   pAUC: 0.5262 ± 0.0051   Recall: 0.0727 ± 0.0025   TN: 854198.0   FP: 463.0   FN: 184.0   TP: 14.0   Params: {'max_samples': 128, 'n_estimators': 10}
 Training Time: 191.3416   Inference Time: 31.0073   pAUC: 0.5229 ± 0.0033   Recall: 0.0758 ± 0.0000   TN: 854160.0   FP: 501.0   FN: 183.0   TP: 15.0   Params: {'max_samples': 128, 'n_estimators': 50}
 Training Time: 332.0520   Inference Time: 56.6222   pAUC: 0.5230 ± 0.0027   Recall: 0.0758 ± 0.0000   TN: 854180.0   FP: 481.0   FN: 183.0   TP: 15.0   Params: {'max_samples': 128, 'n_estimators': 100}
 Training Time: 15.9394   Inference Time: 5.4635    pAUC: 0.5205 ± 0.0046   Recall: 0.0414 ± 0.0038   TN: 854323.0   FP: 338.0   FN: 190.0   TP: 

# 6/6 - Features: activity, hour_sin, hour_cos, weekday_sin, weekday_cos

Testing 24 different hyperparameter combinations
 Training Time: 17.8015   Inference Time: 6.1778    pAUC: 0.6370 ± 0.0136   Recall: 0.1747 ± 0.0104   TN: 853576.0   FP: 1085.0   FN: 165.0   TP: 33.0   Params: {'max_samples': 128, 'n_estimators': 1}
 Training Time: 129.3884   Inference Time: 11.7239   pAUC: 0.6919 ± 0.0157   Recall: 0.3545 ± 0.0299   TN: 849911.0   FP: 4750.0   FN: 120.0   TP: 78.0   Params: {'max_samples': 128, 'n_estimators': 10}
 Training Time: 209.4937   Inference Time: 34.4195   pAUC: 0.7033 ± 0.0084   Recall: 0.3778 ± 0.0262   TN: 849738.0   FP: 4923.0   FN: 114.0   TP: 84.0   Params: {'max_samples': 128, 'n_estimators': 50}
 Training Time: 389.3289   Inference Time: 65.0250   pAUC: 0.7003 ± 0.0082   Recall: 0.3808 ± 0.0126   TN: 849717.0   FP: 4944.0   FN: 120.0   TP: 78.0   Params: {'max_samples': 128, 'n_estimators': 100}
 Training Time: 18.2732   Inference Time: 6.3868    pAUC: 0.6850 ± 0.0152   Recall: 0.2354 ± 0.0380   TN: 852727.0   FP: 1934.0   FN: 144.0 

# Training - LODA

In [ ]:
# Local configuration
classifier_name = "loda"

# Define the classifier that will be used for training
classifier = LODA(
    contamination=CONTAMINATION
)

# Define the hyperparameters grid that will be tested for best results
parameters = {
    "n_bins": [6, 8, 10, 12, 14, 16, 20],
    "n_random_cuts": [25, 50, 75, 100]
}

# Start the training
initiate_training_run(classifier_name, classifier, parameters)

# 1/6 - Features: activity, hour

Testing 28 different hyperparameter combinations
 Training Time: 19.2632   Inference Time: 6.8424    pAUC: 0.4899 ± 0.0000   Recall: 0.0051 ± 0.0000   TN: 846463.0   FP: 8198.0   FN: 197.0   TP: 1.0   Params: {'n_bins': 6, 'n_random_cuts': 25}
 Training Time: 24.3062   Inference Time: 7.7741    pAUC: 0.4948 ± 0.0000   Recall: 0.0152 ± 0.0000   TN: 846185.0   FP: 8476.0   FN: 195.0   TP: 3.0   Params: {'n_bins': 6, 'n_random_cuts': 50}
 Training Time: 30.4371   Inference Time: 8.8710    pAUC: 0.4857 ± 0.0000   Recall: 0.0051 ± 0.0000   TN: 845723.0   FP: 8938.0   FN: 197.0   TP: 1.0   Params: {'n_bins': 6, 'n_random_cuts': 75}
 Training Time: 36.7802   Inference Time: 10.0707   pAUC: 0.4966 ± 0.0000   Recall: 0.0202 ± 0.0000   TN: 845949.0   FP: 8712.0   FN: 194.0   TP: 4.0   Params: {'n_bins': 6, 'n_random_cuts': 100}
 Training Time: 17.5842   Inference Time: 6.2604    pAUC: 0.5077 ± 0.0000   Recall: 0.0152 ± 0.0000   TN: 846402.0   FP: 8259.0   FN: 195.0   TP: 3.0   Params: {'n_bins':

# 2/6 - Features: activity, weekday

Testing 28 different hyperparameter combinations
 Training Time: 17.5614   Inference Time: 6.2148    pAUC: 0.5019 ± 0.0000   Recall: 0.0101 ± 0.0000   TN: 845632.0   FP: 9029.0   FN: 196.0   TP: 2.0   Params: {'n_bins': 6, 'n_random_cuts': 25}
 Training Time: 23.8881   Inference Time: 7.5502    pAUC: 0.4989 ± 0.0000   Recall: 0.0000 ± 0.0000   TN: 846061.0   FP: 8600.0   FN: 198.0   TP: 0.0   Params: {'n_bins': 6, 'n_random_cuts': 50}
 Training Time: 30.3456   Inference Time: 8.7391    pAUC: 0.4918 ± 0.0000   Recall: 0.0000 ± 0.0000   TN: 846244.0   FP: 8417.0   FN: 198.0   TP: 0.0   Params: {'n_bins': 6, 'n_random_cuts': 75}
 Training Time: 36.6002   Inference Time: 9.9500    pAUC: 0.4996 ± 0.0000   Recall: 0.0051 ± 0.0000   TN: 846146.0   FP: 8515.0   FN: 197.0   TP: 1.0   Params: {'n_bins': 6, 'n_random_cuts': 100}
 Training Time: 17.9045   Inference Time: 6.4094    pAUC: 0.5272 ± 0.0000   Recall: 0.0354 ± 0.0000   TN: 846202.0   FP: 8459.0   FN: 191.0   TP: 7.0   Params: {'n_bins':

# 3/6 - Features: activity, hour, weekday

...

Testing 28 different hyperparameter combinations
 Training Time: 18.6556   Inference Time: 6.7306    pAUC: 0.5297 ± 0.0000   Recall: 0.0253 ± 0.0000   TN: 846019.0   FP: 8642.0   FN: 193.0   TP: 5.0   Params: {'n_bins': 6, 'n_random_cuts': 25}
 Training Time: 25.3559   Inference Time: 8.1164    pAUC: 0.5155 ± 0.0000   Recall: 0.0051 ± 0.0000   TN: 846086.0   FP: 8575.0   FN: 197.0   TP: 1.0   Params: {'n_bins': 6, 'n_random_cuts': 50}
 Training Time: 31.9140   Inference Time: 9.4447    pAUC: 0.5075 ± 0.0000   Recall: 0.0152 ± 0.0000   TN: 846171.0   FP: 8490.0   FN: 195.0   TP: 3.0   Params: {'n_bins': 6, 'n_random_cuts': 75}
 Training Time: 38.3774   Inference Time: 10.6614   pAUC: 0.5203 ± 0.0000   Recall: 0.0051 ± 0.0000   TN: 846035.0   FP: 8626.0   FN: 197.0   TP: 1.0   Params: {'n_bins': 6, 'n_random_cuts': 100}
 Training Time: 18.6524   Inference Time: 6.6628    pAUC: 0.5508 ± 0.0000   Recall: 0.0152 ± 0.0000   TN: 846095.0   FP: 8566.0   FN: 195.0   TP: 3.0   Params: {'n_bins':

# 5/6 - Features: activity, weekday_sin, weekday_cos

Testing 28 different hyperparameter combinations
 Training Time: 18.5193   Inference Time: 6.5851    pAUC: 0.5003 ± 0.0000   Recall: 0.0101 ± 0.0000   TN: 846086.0   FP: 8575.0   FN: 196.0   TP: 2.0   Params: {'n_bins': 6, 'n_random_cuts': 25}
 Training Time: 25.6019   Inference Time: 8.2783    pAUC: 0.5007 ± 0.0000   Recall: 0.0101 ± 0.0000   TN: 846012.0   FP: 8649.0   FN: 196.0   TP: 2.0   Params: {'n_bins': 6, 'n_random_cuts': 50}
 Training Time: 31.5822   Inference Time: 9.2458    pAUC: 0.4892 ± 0.0000   Recall: 0.0000 ± 0.0000   TN: 846354.0   FP: 8307.0   FN: 198.0   TP: 0.0   Params: {'n_bins': 6, 'n_random_cuts': 75}
 Training Time: 39.0919   Inference Time: 10.7504   pAUC: 0.5016 ± 0.0000   Recall: 0.0000 ± 0.0000   TN: 846047.0   FP: 8614.0   FN: 198.0   TP: 0.0   Params: {'n_bins': 6, 'n_random_cuts': 100}
 Training Time: 19.3714   Inference Time: 6.8721    pAUC: 0.5396 ± 0.0000   Recall: 0.0909 ± 0.0000   TN: 846150.0   FP: 8511.0   FN: 180.0   TP: 18.0   Params: {'n_bins'

# 6/6 - Features: activity, hour_sin, hour_cos, weekday_sin, weekday_cos

Testing 28 different hyperparameter combinations
 Training Time: 19.9146   Inference Time: 7.1364    pAUC: 0.4832 ± 0.0000   Recall: 0.0051 ± 0.0000   TN: 846111.0   FP: 8550.0   FN: 197.0   TP: 1.0   Params: {'n_bins': 6, 'n_random_cuts': 25}
 Training Time: 26.3382   Inference Time: 8.5763    pAUC: 0.5170 ± 0.0000   Recall: 0.0101 ± 0.0000   TN: 846145.0   FP: 8516.0   FN: 196.0   TP: 2.0   Params: {'n_bins': 6, 'n_random_cuts': 50}
 Training Time: 33.9894   Inference Time: 10.1487   pAUC: 0.5196 ± 0.0000   Recall: 0.0253 ± 0.0000   TN: 846154.0   FP: 8507.0   FN: 193.0   TP: 5.0   Params: {'n_bins': 6, 'n_random_cuts': 75}
 Training Time: 39.7771   Inference Time: 11.3556   pAUC: 0.5132 ± 0.0000   Recall: 0.0202 ± 0.0000   TN: 846142.0   FP: 8519.0   FN: 194.0   TP: 4.0   Params: {'n_bins': 6, 'n_random_cuts': 100}
 Training Time: 19.9505   Inference Time: 7.1661    pAUC: 0.5292 ± 0.0000   Recall: 0.0202 ± 0.0000   TN: 846055.0   FP: 8606.0   FN: 194.0   TP: 4.0   Params: {'n_bins':

# Training - COPOD

In [ ]:
# Local configuration
classifier_name = "copod"

# Define the classifier that will be used for training
classifier = COPOD(
    contamination=CONTAMINATION
)

# Define the hyperparameters grid that will be tested for best results
parameters = {}

# Start the training
initiate_training_run(classifier_name, classifier, parameters)

# 1/6 - Features: activity, hour

Testing 1 different hyperparameter combinations
 Training Time: 12.8109   Inference Time: 6.9620    pAUC: 0.8514 ± 0.0000   Recall: 0.6111 ± 0.0000   TN: 846201.0   FP: 8460.0   FN: 77.0   TP: 121.0   Params: {}

Best hyperparameters:
Params: {}
pAUC:   0.8514 ± 0.0000
Recall: 0.6111 ± 0.0000


# 2/6 - Features: activity, weekday

Testing 1 different hyperparameter combinations
 Training Time: 12.7226   Inference Time: 6.8527    pAUC: 0.5196 ± 0.0000   Recall: 0.0707 ± 0.0000   TN: 846196.0   FP: 8465.0   FN: 184.0   TP: 14.0   Params: {}

Best hyperparameters:
Params: {}
pAUC:   0.5196 ± 0.0000
Recall: 0.0707 ± 0.0000


# 3/6 - Features: activity, hour, weekday

...

Testing 1 different hyperparameter combinations
 Training Time: 13.5957   Inference Time: 7.5458    pAUC: 0.8768 ± 0.0000   Recall: 0.6465 ± 0.0000   TN: 846240.0   FP: 8421.0   FN: 70.0   TP: 128.0   Params: {}

Best hyperparameters:
Params: {}
pAUC:   0.8768 ± 0.0000
Recall: 0.6465 ± 0.0000


# 5/6 - Features: activity, weekday_sin, weekday_cos

Testing 1 different hyperparameter combinations
 Training Time: 13.8348   Inference Time: 7.7476    pAUC: 0.5133 ± 0.0000   Recall: 0.0556 ± 0.0000   TN: 846198.0   FP: 8463.0   FN: 187.0   TP: 11.0   Params: {}

Best hyperparameters:
Params: {}
pAUC:   0.5133 ± 0.0000
Recall: 0.0556 ± 0.0000


# 6/6 - Features: activity, hour_sin, hour_cos, weekday_sin, weekday_cos

Testing 1 different hyperparameter combinations
 Training Time: 14.9633   Inference Time: 9.0336    pAUC: 0.8667 ± 0.0000   Recall: 0.6667 ± 0.0000   TN: 846275.0   FP: 8386.0   FN: 66.0   TP: 132.0   Params: {}

Best hyperparameters:
Params: {}
pAUC:   0.8667 ± 0.0000
Recall: 0.6667 ± 0.0000


# Training - ECOD

In [ ]:
# Local configuration
classifier_name = "ecod"

# Define the classifier that will be used for training
classifier = ECOD(
    contamination=CONTAMINATION,
    n_jobs=1
)

# Define the hyperparameters grid that will be tested for best results
parameters = {}

# Start the training
initiate_training_run(classifier_name, classifier, parameters)

# 1/6 - Features: activity, hour

Testing 1 different hyperparameter combinations
 Training Time: 15.6807   Inference Time: 8.3584    pAUC: 0.8759 ± 0.0000   Recall: 0.7727 ± 0.0000   TN: 846284.0   FP: 8377.0   FN: 45.0   TP: 153.0   Params: {}

Best hyperparameters:
Params: {}
pAUC:   0.8759 ± 0.0000
Recall: 0.7727 ± 0.0000


# 2/6 - Features: activity, weekday

Testing 1 different hyperparameter combinations
 Training Time: 13.4224   Inference Time: 7.0630    pAUC: 0.5245 ± 0.0000   Recall: 0.0758 ± 0.0000   TN: 846184.0   FP: 8477.0   FN: 183.0   TP: 15.0   Params: {}

Best hyperparameters:
Params: {}
pAUC:   0.5245 ± 0.0000
Recall: 0.0758 ± 0.0000


# 3/6 - Features: activity, hour, weekday

...

Testing 1 different hyperparameter combinations
 Training Time: 13.9400   Inference Time: 7.6747    pAUC: 0.8936 ± 0.0000   Recall: 0.7828 ± 0.0000   TN: 846309.0   FP: 8352.0   FN: 43.0   TP: 155.0   Params: {}

Best hyperparameters:
Params: {}
pAUC:   0.8936 ± 0.0000
Recall: 0.7828 ± 0.0000


# 5/6 - Features: activity, weekday_sin, weekday_cos

Testing 1 different hyperparameter combinations
 Training Time: 13.4703   Inference Time: 7.2817    pAUC: 0.5126 ± 0.0000   Recall: 0.0404 ± 0.0000   TN: 846167.0   FP: 8494.0   FN: 190.0   TP: 8.0   Params: {}

Best hyperparameters:
Params: {}
pAUC:   0.5126 ± 0.0000
Recall: 0.0404 ± 0.0000


# 6/6 - Features: activity, hour_sin, hour_cos, weekday_sin, weekday_cos

Testing 1 different hyperparameter combinations
 Training Time: 14.8419   Inference Time: 8.7352    pAUC: 0.8871 ± 0.0000   Recall: 0.6919 ± 0.0000   TN: 846249.0   FP: 8412.0   FN: 61.0   TP: 137.0   Params: {}

Best hyperparameters:
Params: {}
pAUC:   0.8871 ± 0.0000
Recall: 0.6919 ± 0.0000


# Training - CBLOF

In [ ]:
# Local configuration
classifier_name = "cblof"

# Define the classifier that will be used for training
classifier = CBLOF(
    contamination=CONTAMINATION,
    n_jobs=N_JOBS
)

# Define the hyperparameters grid that will be tested for best results
parameters = {
    "n_clusters": [2, 3],
    "alpha": [0.2, 0.4, 0.6, 0.8, 0.9],
    "beta": [2, 4, 8, 16],
    "use_weights": [True, False]
}

# Start the training
initiate_training_run(classifier_name, classifier, parameters)

# Training - PCA

In [ ]:
# Local configuration
classifier_name = "pca"

# Define the classifier that will be used for training
classifier = PCA(
    contamination=CONTAMINATION
)

# Define the hyperparameters grid that will be tested for best results
parameters = {
    "n_components": [1, 2, 3, 4, 5, 6, 7],
    "whiten": [True, False],
    "svd_solver": ["full", "arpack", "randomized"],
    "weighted": [True, False],
    "standardization": [True, False]
}

# Start the training
initiate_training_run(classifier_name, classifier, parameters)

# Training - AE

In [ ]:
# Local configuration
classifier_name = "auto_encoder"

# Define the classifier that will be used for training
classifier = AutoEncoder(
    output_activation="sigmoid",
    optimizer=keras.optimizers.Adam(),
    epochs=100,
    batch_size=16384,
    validation_size=0.1,
    dropout_rate=0.2,
    l2_regularizer=0.1,
    preprocessing=False,
    verbose=0,
    callbacks=[
        keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.2, patience=3, min_lr=1e-6),
        keras.callbacks.EarlyStopping(monitor="val_loss", patience=6)
    ],
    contamination=CONTAMINATION
)

# Define the hyperparameters grid that will be tested for best results
parameters = {
    "hidden_neurons": [[8, 4, 4, 8], [4, 2, 2, 4], [2, 1, 1, 2]],
    "hidden_activation": ["relu", "sigmoid", "tanh"]
}

# Start the training
initiate_training_run(classifier_name, classifier, parameters)

# 5/1 - Features: activity, weekday_sin, weekday_cos

Testing 9 different hyperparameter combinations
 Training Time: 4280.8499   Inference Time: 103.0526   pAUC: 0.5292 ± 0.0000   Recall: 0.0152 ± 0.0000   TN: 846117.0   FP: 8544.0   FN: 195.0   TP: 3.0   Params: {'hidden_activation': 'relu', 'hidden_neurons': [8, 4, 4, 8]}
 Training Time: 4117.9914   Inference Time: 100.8420   pAUC: 0.5109 ± 0.0000   Recall: 0.0101 ± 0.0000   TN: 846182.0   FP: 8479.0   FN: 196.0   TP: 2.0   Params: {'hidden_activation': 'relu', 'hidden_neurons': [4, 2, 2, 4]}
 Training Time: 4280.2652   Inference Time: 97.3688   pAUC: 0.5072 ± 0.0000   Recall: 0.0152 ± 0.0000   TN: 846140.0   FP: 8521.0   FN: 195.0   TP: 3.0   Params: {'hidden_activation': 'relu', 'hidden_neurons': [2, 1, 1, 2]}
 Training Time: 4474.6694   Inference Time: 105.4690   pAUC: 0.5306 ± 0.0000   Recall: 0.0152 ± 0.0000   TN: 846049.0   FP: 8612.0   FN: 195.0   TP: 3.0   Params: {'hidden_activation': 'sigmoid', 'hidden_neurons': [8, 4, 4, 8]}
 Training Time: 5320.7489   Inference Time: 111.71

# Training - Deep SVDD

In [ ]:
# Local configuration
classifier_name = "deep_svdd"

# Define the classifier that will be used for training
classifier = DeepSVDD(
    output_activation="sigmoid",
    optimizer=keras.optimizers.Adam(),
    epochs=100,
    batch_size=16384,
    validation_size=0.1,
    dropout_rate=0.2,
    l2_regularizer=0.1,
    preprocessing=False,
    verbose=0,
    callbacks=[
        keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.2, patience=3, min_lr=1e-6),
        keras.callbacks.EarlyStopping(monitor="val_loss", patience=6)
    ],
    contamination=CONTAMINATION
)

# Define the hyperparameters grid that will be tested for best results
parameters = {
    "hidden_neurons": [[64, 32], [32, 16], [16, 8], [8, 4], [4, 2], [2, 1]],
    "hidden_activation": ["relu", "sigmoid", "tanh"],
    "use_ae": [True, False]
}

# Start the training
initiate_training_run(classifier_name, classifier, parameters)